# Investigation of Python data visualization tools

## Introduction

See the very useful 3-part post on the [Anaconda website](https://www.anaconda.com/python-data-visualization-2018-why-so-many-libraries/)

![python-vis-landscape](https://www.anaconda.com/wp-content/uploads/2019/01/PythonVisLandscape.jpg)

An almost complete list of packages can also be found [here](https://wiki.python.org/moin/NumericAndScientific/Plotting)

## Matplotlib (https://matplotlib.org/3.1.1/index.html)

`matplotlib` is by far the most popular and widely used package in the scientific data visualization community. It is an old (started in 2003) and very mature project, and yet still sees very active development.
It is the go-to library for creating publishable vector graphics pdf figures in articles, and specialises in 2D plots.

It is either already installed on most systems, or very easy to install on all platforms. Its strength is on high-quality static figures, and is behind other projects on the interactivity side of things.

It has some 3D capabilities through the `Axes3D`.

To see what `matplotlib` can do, a good place to go is [its sample plots page](https://matplotlib.org/tutorials/introductory/sample_plots.html).

A small example is shown below, to illustrate both the syntax and the output generated.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
N = 50
x = np.arange(N)
y = np.arange(N)
z = np.random.rand(N, N)

fig, ax = plt.subplots(2, 2)
ax[0][0].plot(x, z[:, 0])
ax[0][0].set_title("1D line plot")
ax[0][1].hist(z[:, 0], bins=20)
ax[0][1].set_title("1D histogram")
ax[1][0].imshow(z)
ax[1][0].set_title("2D image")
ax[1][1].scatter(z[:, 0], z[:, 1], c=z[:, 2])
ax[1][1].set_title("Scatter plot")
fig.show()

### Interface

When talking about `matplotlib`, a common complaint that arises is that the API is a little difficult to get around, especially for a non-experienced user/programmer. This is somewhat true, but in some sense a little inevitable in a highly customizable tool. It is however relatively straightforward to hide the interface from the user by creating pre-made wrapper functions that will cover 90% of the cases the users are after.

Examples of this are very common, for example the `pandas` library uses `matplotlib` for creating plots from the contents of its data frames, via a simple `.plot` method.

### Interactivity

Another common misconception is that interactivity is virtually not supported in `matplotlib`. In fact, the library allows multiple interactivity options with mouse events (scrolling, clicking), as well as via its own set of `widgets`. These include sliders, radio buttons, check boxes, etc... and are a nice alternative to `ipywidgets`, guaranteed to work out-of-the-box with matplotlib figures.

See a [diagram](https://matplotlib.org/3.1.0/_images/inheritance-9d71a95f6d3ec40d1246549117ad7959f7b88c66.png) listing all the available widgets.

Interactivity is available both in the default Qt render window, and in the Jupyter notebook. Note that for the latter, a magic command `%matplotlib notebook` needs to be added **in a separate cell** at the start of the notebook, otherwise only static images are produced. Hiding this cell from the notebook inside a library import (to save the user from having to enter it in every new notebook) is apparently not yet possible. 

[A few noteable examples of interactive plots](examples/matplotlib.ipynb#interactive_examples)

### Performance

`matplotlib` is very fast, especially for generating images with regularly-sized pixels via its `imshow` function which is much faster than other web-based solutions listed in this report.

Saving figures to vector graphics (pdf) can however sometimes be quite slow.

Interactivity with plots shows mixed performance. Zooming onto lines or images is fast, but interaction with widgets feels sluggish. It has to be noted that interaction is smoother inside the `Qt` window compared to the Jupyter notebook. Maybe some emulation is taking place to render in the notebook, thus degrading performance?

### Documentation
The documentation is very extensive, well written, with a complete API reference and examples galore.

### Community
The user base of `matplotlib` is unrivaled, and the variety of examples found around the web is the richest out there. T

### Risks and project sustainability

![mpl pulse](images/pulse_matplotlib.png)

The project has been around for many years, and will still be there for many to come. No real risks are associated with such a well established library. 

### Standard use cases
See [here](examples/matplotlib.ipynb) for a notebook demonstrating how some standard use cases are achieved using `matplotlib`.

### Summary

<span style="color:green">**Pros:**</span>

- Widespread availability on existing systems
- Easily installable
- Large well-maintained project with many developers

<span style="color:red">**Cons:**</span>

---

## Plotly (https://plot.ly/python/)

`plotly` is a very popular library for creating highly interactive plots to be used in a web-browser. It is Javascript based, but also has a complete Python API, making it well suited for Jupyter notebook use.

It boasts a high performance, especially for scatter plots, by making use of the WebGL technology (see [here](https://plot.ly/python/webgl-vs-svg/)).

One very nice feature of `plotly` is that its output is standalone html code that can be embedded in any web-site.

The same example as for `matplotlib` is shown below

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Scatter(x=x, y=z[:, 0]), row=1, col=1)
fig.add_trace(go.Histogram(x=z[:, 0], xbins={"size": 0.05}), row=1, col=2)
fig.add_trace(go.Heatmap(z=z, showscale=False), row=2, col=1)
fig.add_trace(go.Scatter(x=z[:, 0], y=z[:, 1], mode='markers',
    marker={"color": z[:, 2]}), row=2, col=2)

fig.show()

### Dash

The makers of `plotly` also offer a platform called `Dash` which aims to make it very easy to create web apps with interactive menus, buttons, sliders and plots. Examples can be found here: https://dash-gallery.plotly.host/Portal/, and some are indeed very impressive.

It is also now possible to embed `Dash` apps as a separate window in the JupyterLab environment, to allow both clickable and scriptable interaction with an app: https://github.com/plotly/jupyterlab-dash

This could be a very nice alternative to developing GUIs for something like interfaces in `Mantid`.

### Interactivity

Interactivity is varied and extremely well supported in `plotly`. It is one of the strengths of the library, and it feels like much work has gone into making this their selling point.

Zooming and panning is smooth, and connecting graphs to Jupyter `ipywidgets` works well. It is even possible to go further with selection events, whereby selecting points in a plot can affect what data is displayed in another plot or a table. See [here](https://plot.ly/python/selection-events/) for an exmaple.

### Performance

`plotly`'s performance is very good for line/scatter plots, even for very large datasets (1M points), via the `Scattergl` method.

Performance for 3D visualization seems reasonable.

### Documentation

### Community
A large community uses `plotly`.
Developers are responsive to posts on the forum, and seem very happy to help.

### Risks and project sustainability

![plotly pulse](images/pulse_plotly.png)

### Standard use cases
See [here](examples/plotly.ipynb) for a notebook with the `plotly` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Easy to install
- Interactivity &starf;&starf;&starf;&starf;&starf;
- Large well-maintained project
- Very complete in terms of features
- 3D capable

<span style="color:red">**Cons:**</span>

- Not easily useable outside of a web environment. It is possible to use a custom renderer and open a `plotly`
  graph inside a Qt window (see [here](examples/plotly.ipynb#plotly_in_qt))

---

## PyQtGraph (http://www.pyqtgraph.org/)

`pyqtgraph` is a very impressive tool based on OpenGL that provides 1D-3D data visualization with astonishing performance. Zooming in on a line made up of millions of points is flawless, and the same goes for a 5000x5000 image.

The OpenGL shaders can be used to create beautiful effects in 3D environments.

The installation is simple via `pip` or `conda` and comes with a large set of useful examples.

### Interactivity

Interaction with graphs via mouse events is extremely smooth, even for large datasets. It is also possible to define regions-of-interest which can be moved/reshaped with the mouse.

As the name suggests, `pyqtgraph` integrates very well in `Qt` windows, and it is relatively simple to connect graphs to sliders, buttons to enhance interactivity.

### Performance

### Documentation

### Community

### Risks and project sustainability

![pyqtgraph pulse](images/pulse_pyqtgraph.png)

The project however does have some major drawbacks.

#### Lone developer

`pyqtgraph` is only being developed by one person, Luke Campagnola, and development has been slow for the last ~2 years. There are 200+ issues open on github and 100 PRs, including some that are several years old. While it seems many different people are creating PRs, all of them seem to fail the CI tests. However, they are still getting merged regardless. There is a real doubt as to whether this project will still be alive in 5 years.

#### Use in Jupyter has limitations

When using `pyqtgraph` in a Jupyter notebook, one cannot embed the plots in the notebook, a separate `Qt` window has to be spawned. This causes problems if running on a remote machine, as well as making it difficult to remember which plot corresponded to which cell in a lengthy notebook.

In addition, when spawning a `Qt` window via the
```Python
app = QtGui.QApplication([])
QtGui.QApplication.exec_()
```
method, the cell in the notebook is locked and everything below can only be executed once the plot window has been killed, making the use of Jupyter rather pointless. This can however be alleviated via the use of the magic command
```Python
%gui qt
```

#### 3D visualization is not fully developed

While representing scatter plots, surfaces, etc... works very well in 3D, it feels like development was never fully completed. For instance, there are no annotated axes for 3D plots. Only a mesh that serves as a point of reference for the eye is available, with no tick marks nor any labels/annotations. While it may not be too difficult to add, it is not obvious development would be swift, judging by the number of open PRs.

### Standard use cases
See [here](examples/pyqtgraph.ipynb) for a notebook with the `pyqtgraph` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Easy to install
- Lightning fast performance
- Very complete in terms of features, including clickable/dragable points/lines
- Good integration with `Qt`
- 3D capable

<span style="color:red">**Cons:**</span>

- Only one developer
- Cannot embed plots in Jupyter notebook
- 3D visualization is not fully developed (missing axes)

---

## Bqplot

`bqplot` is a relatively new library (started 2015), that is developed by the makers of Jupyter, `ipywidgets`, `ipyvolume`, and `glueviz`. It is targetted at Jupyter users. It is built on `ipywidgets`, and every item in the graphs are a widget, making plots highly configurable with many interaction possibilities.

In [ ]:
from bqplot import pyplot as plt

fig1 = plt.figure()
line = plt.plot(x, z[:, 0])
plt.show()
fig2 = plt.figure()
hist = plt.hist(z[:, 0], bins=20)
plt.show()
fig3 = plt.figure()
heatmap = plt.heatmap(z, x=x, y=y)
plt.show()
fig4 = plt.figure()
scat = plt.scatter(z[:, 0], z[:, 1])
plt.show()

### Interactivity

Interactivity is at the heart of `bqplot`.
It is, for example, possible to drag individual points in a scatter plot.

In [ ]:
## Enabling moving of points in scatter. Try to click and drag any of the points in the scatter and 
## notice the line representing the mean of the data update
fig = plt.figure()
scat = plt.scatter(z[:, 0], z[:, 1], colors=['orange'], enable_move=True)
lin = plt.plot([], [], line_style='dotted', colors=['orange'])

def update_line(change=None):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]

update_line()

# update line on change of x or y of scatter
scat.observe(update_line, names=['x'])
scat.observe(update_line, names=['y'])

fig

One can also create linked plots where data selections in one graph can affect the data displayed in another plot. This in fact can be made to interact with any `ipywidgets` based entity, as is illustrated in [this](https://ipyvolume.readthedocs.io/en/latest/bqplot.html#bqplot-scatter-plot) example with `ipyvolume`.

### Performance

Performance is the achilles heel of `bqplot`. The time for rendering graphs is similar to other projects, but interaction feels very slow. Just having 5 histograms on the same plot makes zooming in and out painfully slow: 

In [ ]:
N = 50
M = 5
x = np.arange(N)
y = np.random.rand(N, M)
fig = plt.figure()
for i in range(M):
    plt.bar(x, y[:, i], padding=0.0, color=[i]*N)
plt.show()

There are many issues on the `github` repository on the topic of performance, and this is obviously an area of very active development. In particular it appears that they are trying to make use of WebGL more and more (see [here](https://github.com/glue-viz/bqplot-image-gl)), which would clearly help. Maybe it is simply a matter of time before performance is on par with other similar projects? It is worth coming back to `bqplot` in a year's time to see how this has developed.

### Documentation

Documentation is rather poor on the [readthedocs](https://bqplot.readthedocs.io/en/latest/index.html) website. It basically contains the auto-generated API reference and not much more.

To get to grips with how to use `bqplot`, one has to use the (actually very useful) Jupyter notebooks that are bundled up into a [binder](https://mybinder.org/v2/gh/bloomberg/bqplot/0.11.x?filepath=examples/Index.ipynb) image.

### Community

### Risks and project sustainability
![bqplot pulse](images/pulse_bqplot.png)

The team of `bqplot` developers comprise, among others, Sylvain Corlay (Quantstack founder) & Maarten Breddels (ipyvolume creator), and they are part of several collaborations that make use of `bqplot`, such as [`glue-jupyter`](https://github.com/glue-viz/glue-jupyter), a Jupyter version of the `glueviz` project by Thomas Robitaille (who is also a core `astropy` contributor).

Development appears to have been steady over the past 5 years, but it also seems to happens in several different places. Some changes are for instance first implemented in `glue-viz/bqplot-image-gl`, before they are integrated in the main repository. This could be both a good and bad thing.

### Summary

<span style="color:green">**Pros:**</span>

- Easy to install
- Team of very capable developers
- Very complete in terms of features, including clickable/dragable points/lines
- Good integration with Jupyter
- 3D capable

<span style="color:red">**Cons:**</span>

- No 3D (relies on `ipyvolume` for 3D)

## Holoviz

## QtCharts

## Ipyvolume

## Pyvista

## Altair


`Altair` is also capable of linking data in different plot, reacting through selection of points. As shown in [this](https://altair-viz.github.io/gallery/scatter_with_histogram.html) example, one can select a range in a histogram and only the corresponding points inside the selected bins will appear in the scatter plot.

This is an increasingly popular data visualization technique embraced for instance by the `glueviz` project and many others.

### Risks and project sustainability

One of the main disadvantages of `Altair` is that it requires the data to be in a `pandas` DataFrame, thus either forcing us to use `pandas` as our data container, or requiring a (potentially costly?) conversion step between our data container and `pandas` for the plotting.